In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [23]:
data_age = pd.read_csv('data/cleaned_age.csv')
data_health = pd.read_csv('data/cleaned_health.csv')
data_heat = pd.read_csv('data/cleaned_heat.csv')
data_depr = pd.read_csv('data/cleaned_depr.csv')
data_hhold = pd.read_csv('data/cleaned_hhold.csv')
data_ethgrp = pd.read_csv('data/cleaned_ethgrp.csv')
data_cars = pd.read_csv('data/cleaned_cars.csv')

meta_data = pd.read_csv('data/meta_all.csv')

lsoa_data = pd.read_csv('manchester_data_la.csv', sep = ',')

In [4]:
data_depr.head()

,GEO_CODE,F997,F998,F999,F1000,F1001
0,E01004766,279,279,196,39,2
1,E01004767,343,255,134,33,3
2,E01004768,388,166,46,5,0
3,E01004769,380,210,84,10,1
4,E01004770,279,217,106,13,1


In [5]:
def prop_of_tot(old_data):
    
    data = old_data.set_index('GEO_CODE').astype(int)
    data['sum'] = data.sum(axis = 1, skipna = all)
    
    new_df = pd.DataFrame(index = data.index, columns = data.columns)
    for i in range(len(data)):
        for j in range(int(len(data.columns) - 1)):
            new_df.iloc[i][j] = (data.iloc[i][j] / data.iloc[i][int(len(data.columns) - 1)])
    
    new_df = new_df.dropna(axis = 1)

    return(new_df)

In [6]:
prop_health = prop_of_tot(data_health)
prop_heat = prop_of_tot(data_heat)
prop_depr = prop_of_tot(data_depr)
prop_cars = prop_of_tot(data_cars)

In [7]:
data_ethgrp = data_ethgrp.set_index('GEO_CODE')

for i in range(len(data_ethgrp)):
    for j in range(len(data_ethgrp.columns)):
        data_ethgrp.iloc[i][j] = data_ethgrp.iloc[i][j] ** 2

data_ethgrp['hhi'] = data_ethgrp.sum(axis = 1, skipna = all)

In [8]:
data_ethgrp = data_ethgrp[['hhi']]

In [9]:
# normalise hhi --> turns to array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(data_ethgrp.values)

C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [10]:
hhi_list = []

for i in range(len(x_scaled)):
    hhi_list.append(float(x_scaled[i]))

In [11]:
data_ethgrp['hhi_norm'] = hhi_list

In [12]:
data_hhold = data_hhold.set_index('GEO_CODE')
data_age = data_age.set_index('GEO_CODE')

In [13]:
prop_hhold = pd.DataFrame(index = data_hhold.index, columns = data_hhold.columns)

for i in range(len(data_hhold)):
    for j in range(1, len(data_hhold.columns)):
        prop_hhold.iloc[i][j] = (data_hhold.iloc[i][j] / data_hhold.iloc[i][0])
        
prop_hhold = prop_hhold.dropna(axis = 1)

In [19]:
# join data

man_data = prop_health.join(prop_heat).join(prop_depr).join(prop_cars).join(prop_hhold).join(data_age).join(data_ethgrp)

tokeep = ['F184', 'F1782', 'F2345', 'F997', 'F1838', 'F991', 'hhi_norm'] 

man_data = (man_data[tokeep].rename(columns = {'F184':'age_mean',
                                               'F1782':'badhealth',
                                               'F2345':'centheat',
                                               'F997':'nodepr',
                                               'F1838':'studenthhold',
                                               'F991':'nocars',
                                               'hhi_norm':'eth_hhi'})\
           .astype(np.float))

In [24]:
lsoa_data = lsoa_data.set_index('geo_code_lsoa')

In [25]:
all_data = lsoa_data[['local_auth']].join(man_data)

In [26]:
all_data.head()

,local_auth,age_mean,badhealth,centheat,nodepr,studenthhold,nocars,eth_hhi
geo_code_lsoa,,,,,,,,
E01005268,Manchester,40.40783,0.042121,0.976861,0.509054,0.001006,0.280684,0.285152
E01033656,Manchester,22.29914,0.003438,0.909330,0.371879,0.268068,0.674113,0.361815
E01005184,Manchester,29.31636,0.031117,0.966624,0.441592,0.219512,0.410783,0.159142
E01005298,Manchester,30.76403,0.041417,0.973479,0.483619,0.015601,0.260530,0.064515
E01005300,Manchester,36.61435,0.051249,0.977551,0.404082,0.004082,0.159184,0.060058


In [28]:
all_data['centre_dummy'] = np.nan
all_data['centre'] = 'unknown'

for i in range(len(all_data)):
    if all_data['local_auth'][i] == 'Manchester':
        all_data['centre_dummy'][i] = 1
        all_data['centre'][i] = 'Yes'
    else:
        all_data['centre_dummy'][i] = 0
        all_data['centre'][i] = 'No'

C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\gylk

In [30]:
all_data['depr_dummy'] = np.nan

for i in range(len(all_data)):
    if all_data['nodepr'][i] >= 0.5:
        all_data['depr_dummy'][i] = 1
    else:
        all_data['depr_dummy'][i] = 0

C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [31]:
all_data.to_csv('data/manchester_data.csv')